In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import csv
from pyspark.sql import SparkSession
import numpy as np
import seaborn as sns
import json
import matplotlib.pyplot as plt 
from pyspark.sql.types import StructType, StructField, StringType
spark = SparkSession.builder \
    .appName("MySparkApp") \
    .getOrCreate()
import warnings
warnings.filterwarnings("ignore")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/30 19:48:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [26]:
df=spark.read.csv("MLR_train.csv",header=True,inferSchema=True)

In [27]:
df=df.drop(*['Loan_ID','Loan_Status'])

In [28]:
df.show(5)

+------+-------+----------+------------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+
|Gender|Married|Dependents|   Education|Self_Employed|ApplicantIncome|CoapplicantIncome|LoanAmount|Loan_Amount_Term|Credit_History|Property_Area|
+------+-------+----------+------------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+
|  Male|     No|         0|    Graduate|           No|           5849|              0.0|      null|             360|             1|        Urban|
|  Male|    Yes|         1|    Graduate|           No|           4583|           1508.0|       128|             360|             1|        Rural|
|  Male|    Yes|         0|    Graduate|          Yes|           3000|              0.0|        66|             360|             1|        Urban|
|  Male|    Yes|         0|Not Graduate|           No|           2583|           2358.0|       120|             360|        

In [29]:
from pyspark.sql.functions import col
for cols in df.columns:
    print(cols,df.where(col(cols).isNull()).count())

Gender 13
Married 3
Dependents 15
Education 0
Self_Employed 32
ApplicantIncome 0
CoapplicantIncome 0
LoanAmount 22
Loan_Amount_Term 14
Credit_History 50
Property_Area 0


In [30]:
result_df=df

In [31]:
columns_to_drop = []
Percentage = 30

for cols in result_df.columns:
    null_count = result_df.where(col(cols).isNull()).count()
    null_percentage = round(null_count * 100 / result_df.count(), 2)
    #print(cols, result_df.select(cols).count(), null_percentage)
    if null_percentage > Percentage:
        columns_to_drop.append(cols)
        result_df = result_df.drop(cols)
print("Columns to drop:", columns_to_drop)
print("Remaining columns:", len(result_df.columns))
print("Number of rows:", result_df.count())

Columns to drop: []
Remaining columns: 11
Number of rows: 614


In [32]:
df_pd=result_df.toPandas()
df_pd

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban
...,...,...,...,...,...,...,...,...,...,...,...
609,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural
610,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural
611,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban
612,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban


In [33]:
from pyspark.sql import functions as f
Gender_type_counts = result_df.groupBy("Gender").agg(f.count("Gender").alias("count"))
#display(Gender_type_counts.orderBy("count"))

In [34]:
Gender_type_counts.show()

+------+-----+
|Gender|count|
+------+-----+
|  null|    0|
|Female|  112|
|  Male|  489|
+------+-----+



In [35]:
from pyspark.sql.functions import col
result_df.select('Gender').where(col('Gender').isNull()).show()

+------+
|Gender|
+------+
|  null|
|  null|
|  null|
|  null|
|  null|
|  null|
|  null|
|  null|
|  null|
|  null|
|  null|
|  null|
|  null|
+------+



In [36]:
df=df.drop_duplicates()

In [37]:
df.count()

614

In [43]:
df_pd

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban
...,...,...,...,...,...,...,...,...,...,...,...
609,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural
610,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural
611,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban
612,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban


In [76]:
df_pd.dropna(inplace=True)

In [77]:
round(df_pd.corr()*100)

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
ApplicantIncome,100.0,-11.0,50.0,-1.0,-6.0
CoapplicantIncome,-11.0,100.0,19.0,-1.0,-1.0
LoanAmount,50.0,19.0,100.0,5.0,-4.0
Loan_Amount_Term,-1.0,-1.0,5.0,100.0,3.0
Credit_History,-6.0,-1.0,-4.0,3.0,100.0


In [78]:
x=df_pd[['ApplicantIncome','CoapplicantIncome','Loan_Amount_Term','Credit_History']]
y=df_pd['LoanAmount']

In [79]:
x

,ApplicantIncome,CoapplicantIncome,Loan_Amount_Term,Credit_History
1,4583,1508.0,360.0,1.0
2,3000,0.0,360.0,1.0
3,2583,2358.0,360.0,1.0
4,6000,0.0,360.0,1.0
5,5417,4196.0,360.0,1.0
...,...,...,...,...
609,2900,0.0,360.0,1.0
610,4106,0.0,180.0,1.0
611,8072,240.0,360.0,1.0
612,7583,0.0,360.0,1.0


In [80]:
from sklearn.model_selection import train_test_split

In [81]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.2,random_state=21)

In [82]:
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

(384, 4) (96, 4) (384,) (96,)


In [83]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [84]:
x_train=scaler.fit_transform(x_train)

In [85]:
x_test=scaler.transform(x_test)

# Model Training

In [86]:
from sklearn.linear_model import LinearRegression

In [87]:
regression=LinearRegression()

In [88]:
regression

LinearRegression()

In [90]:
#x_train.dropna(inplace=True)
#y_train.dropna(inplace=True)

In [91]:
regression.fit(x_train,y_train)

LinearRegression()

In [92]:
regression.coef_

array([45.66728686, 23.91299676,  3.88137937, -0.46773851])

In [93]:
regression.intercept_

146.2421875

In [94]:
##Prediction

In [95]:
y_pred=regression.predict(x_test)

In [99]:

# Assuming y_test and y_pred are NumPy arrays or lists
data = {'Actual': y_test, 'Predicted': y_pred}
df1 = pd.DataFrame(data)

In [103]:
y_test

131     74.0
20     104.0
66     126.0
121     44.0
273    150.0
       ...  
31      74.0
253    279.0
192    160.0
154     50.0
88     210.0
Name: LoanAmount, Length: 96, dtype: float64

In [104]:
y_pred

array([125.5869204 , 149.33602068, 129.36888412, 120.24061974,
       133.82503865, 149.5128195 , 134.30054154, 114.05196683,
       161.89090756, 109.01172335, 124.10332296, 138.97575839,
       223.69930335, 123.85368711, 155.90869419, 123.4257069 ,
       116.97197231, 217.60890268, 127.55156809, 191.57009561,
       179.01782431, 123.81719679, 192.53516394, 141.35037231,
       127.54912675, 125.1646402 , 130.83287998, 172.2162133 ,
       177.53507443, 132.72732147, 110.12084026,  95.7570213 ,
       135.11115957, 131.42474488, 157.1637467 , 127.65170251,
       136.41906152, 180.88984334, 112.93801016, 123.55491447,
       112.23703508, 123.7377173 , 158.65693458, 120.912613  ,
       332.6122472 , 164.24841341, 110.93357144, 190.2182296 ,
       122.59103189, 109.27060233, 133.80180811, 112.96029844,
       135.59687062, 120.70971325, 130.22114584, 121.58648036,
       123.83909807, 126.47181176, 150.47617334, 386.91815289,
       111.65787185, 123.37405687, 186.11742354, 156.53

In [108]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,mean_absolute_percentage_error,r2_score

In [110]:
mae=mean_absolute_error(y_test,y_pred)
mape=mean_absolute_percentage_error(y_test,y_pred)*100
mse=mean_squared_error(y_test,y_pred)
rmse=np.sqrt(mae)

In [111]:
print(mae,mape, mse,rmse)

43.73190157532258 45.484574387926564 5164.953171763319 6.613010023833517


In [113]:
score=r2_score(y_test,y_pred)

In [114]:
score

-0.08543564341084053

In [115]:
1-(1-score)*(len(y_test)-1)/((len(y_test)-1)-x_test.shape[1]-1)

-0.1457376236003316

In [129]:
# Assuming you have a trained StandardScaler named scaler
input_data = (
    4583,
    1508.0,
    360.0,
    1
)

# Ensure that the input_data has the same number of features as the data used to fit the scaler
scaled_input = scaler.transform([input_data])

# Make predictions using the model
prediction = regression.predict(scaled_input)

# Round the prediction to 2 decimal places
rounded_prediction = round(prediction[0], 2)

print(rounded_prediction)


141.11


In [122]:
df_pd.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban
5,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban
